In [40]:
import os
from ultralytics import YOLO
from shapely.geometry import Polygon
import cv2
import numpy as np
from PIL import Image
import time
from random import randint
import math
from pathlib import Path

In [41]:
test_path = "test_images"
test_output_path = "output_test"

In [42]:
model_path = "viginet_aug_v9_3_240/segment/train/weights/best.pt"

In [43]:
#List all images from directory

def list_dir(path):
  images  = []

  for filename in os.listdir(path):
    # Check if file extension is an image format
    if filename.endswith(('.jpg', '.jpeg', '.png')):
      # Construct the full image path
      image_path = os.path.join(path, filename).replace("\\", "/")
      # Append the path to the list
      images.append(image_path)

  return images

orig_images  = list_dir(test_path)
orig_images.sort()

In [44]:
print(orig_images)

['test_images/Block_1C1_Row_1_Middle_1008.png', 'test_images/Block_1C1_Row_1_Middle_1028.png', 'test_images/Block_1C1_Row_1_Middle_1053.png', 'test_images/Block_1C1_Row_1_Middle_1088.png', 'test_images/Block_1C1_Row_1_Middle_1233.png', 'test_images/Block_1C1_Row_1_Middle_1288.png', 'test_images/Block_1C1_Row_1_Middle_1313.png', 'test_images/Block_1C1_Row_1_Middle_1323.png', 'test_images/Block_1C1_Row_1_Middle_1343.png', 'test_images/Block_1C1_Row_1_Middle_1363.png', 'test_images/Block_1C1_Row_1_Middle_1388.png', 'test_images/Block_1C1_Row_1_Middle_1393.png', 'test_images/Block_1C1_Row_1_Middle_1418.png', 'test_images/Block_1C1_Row_1_Middle_1463.png', 'test_images/Block_1C1_Row_1_Middle_1478.png', 'test_images/Block_1C1_Row_1_Middle_1534.png', 'test_images/Block_1C1_Row_1_Middle_1558.png', 'test_images/Block_1C1_Row_1_Middle_1566.png', 'test_images/Block_1C1_Row_1_Middle_1594.png', 'test_images/Block_1C1_Row_1_Middle_1622.png', 'test_images/Block_1C1_Row_1_Middle_1634.png', 'test_images

In [45]:
model = YOLO(model_path)

In [46]:
print(model.info())

YOLOv9e-seg summary: 1261 layers, 60451891 parameters, 0 gradients, 248.1 GFLOPs
(1261, 60451891, 0, 248.06732799999997)


In [47]:
error_file = []

In [48]:
def test_and_save(orig_image_path):
    predicted_output = model.predict(orig_image_path)


    
    for r in predicted_output:
          img = np.copy(r.orig_img)
          img_name = Path(r.path).stem # source image base-name

          # Create binary mask
          b_mask = np.zeros(img.shape[:2], np.uint8)
          

          # Iterate each object contour (multiple detections)
          for ci,c in enumerate(r):
              #  Get detection class name
              label = c.names[c.boxes.cls.tolist().pop()]
              #  Extract contour result
              contour = c.masks.xy.pop()
              #  Changing the type
              contour = contour.astype(np.int32)
              #  Reshaping
              contour = contour.reshape(-1, 1, 2)


              # Draw contour onto mask
              try:
                predict_mask = cv2.drawContours(b_mask,
                                    [contour],
                                    -1,
                                    (255, 255, 255),
                                    cv2.FILLED)
              except:
                  error_file.append(orig_image_path)
                  return -1
    output_image_path = test_output_path+'/'+orig_image_path.split("/")[-1]
    cv2.imwrite(output_image_path, b_mask)
              
    

In [49]:
# test_and_save("test_images/Block_1C1_Row_1_Middle_728.png")

In [50]:
for i in range(len(orig_images)):
    test_and_save(orig_images[i])


image 1/1 c:\Users\Kaushik Vishwakarma\OneDrive\Desktop\Projects\DLimageseg-aug\test_images\Block_1C1_Row_1_Middle_1008.png: 320x640 21 grapes, 2533.6ms
Speed: 5.0ms preprocess, 2533.6ms inference, 35.0ms postprocess per image at shape (1, 3, 320, 640)

image 1/1 c:\Users\Kaushik Vishwakarma\OneDrive\Desktop\Projects\DLimageseg-aug\test_images\Block_1C1_Row_1_Middle_1028.png: 320x640 21 grapes, 2271.5ms
Speed: 3.0ms preprocess, 2271.5ms inference, 27.0ms postprocess per image at shape (1, 3, 320, 640)

image 1/1 c:\Users\Kaushik Vishwakarma\OneDrive\Desktop\Projects\DLimageseg-aug\test_images\Block_1C1_Row_1_Middle_1053.png: 320x640 13 grapes, 2348.0ms
Speed: 4.0ms preprocess, 2348.0ms inference, 20.0ms postprocess per image at shape (1, 3, 320, 640)

image 1/1 c:\Users\Kaushik Vishwakarma\OneDrive\Desktop\Projects\DLimageseg-aug\test_images\Block_1C1_Row_1_Middle_1088.png: 320x640 10 grapes, 1969.8ms
Speed: 7.0ms preprocess, 1969.8ms inference, 22.0ms postprocess per image at shape (